In [1]:
from sklearn.datasets import load_wine
import matplotlib.pyplot as plt
import numpy as np
import random

In [2]:
data = load_wine()
shuffledIndices = np.random.permutation(len(data.target))
data.data, data.target = data.data[shuffledIndices], data.target[shuffledIndices]

In [3]:
# define Dataset
class Dataset:
    def __init__(self, dataset, vector_machine):
      self.x, self.y = dataset
      self.y = np.array([vector_machine[_] for _ in self.y])
    def __getitem__(self, item): return self.x[item], self.y[item]
    def __len__(self): return len(self.y)
# Dataset

def dataset_visualize(dataset: Dataset):
  for x, y in dataset: print(f"{'-' * 16}\nx: {x}\ny: {y}")
# dataset_visualize

# init Datasets
hv_machine = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
trainset = Dataset((data.data, data.target), vector_machine=hv_machine)
testset = Dataset((data.data, data.target), vector_machine=hv_machine)
###dataset_visualize(trainset)

In [4]:
def sigmoid(x, e=2.7182): return 1 / (1 + e ** (-x))
def relu(x): return np.array([max(0, _) for _ in x])

In [5]:
def gradient(x, y, weight, act_func, lr):
    indications = act_func(np.dot(x, weight))
    return (lr / x.shape[0]) * (indications - y)
# gradient

class Node:
    _next, _prev = None, None
    def __init__(self, n_inpt, n_oupt, act_func=lambda x: x):
        self.weight = np.random.rand(n_inpt, n_oupt)
        self.act_func = act_func
    # __init__

    def __call__(self, x):
        result = np.dot(x, self.weight)
        self.output = result.copy()
        self.grad = gradient(x, self.ouput, self.weight, act_func=self.act_func, lr=lr)
        return result
    # __call__
    
    def train(self, f, h, h_, lr):
        f_ = gradient(h, f, self.weight, self.act_func, lr)
        self.weight -= f_ * h_
    # train
    
    def loss(self, dataset):
        x, y = dataset.x, dataset.y
        probability = np.dot(x, self.weight)
        loss = np.mean(np.exp(1 - probability) * y) + np.sum(np.exp(probability) * (1 - y))
        print(f"cost: {loss:.3f}")
    # loss
# Node

In [6]:
class ANN:
    def __init__(self):
        prev, curr, first, attr = None, None, True, None
        for attr_name in dir(self):
            attr = getattr(self, attr_name)
            if type(attr).__name__ == "Node" and attr_name != "root" and attr_name != "tail":
                prev, curr = curr, attr
                if first is True: self.root, first = curr, False
                curr._prev = prev
                if type(curr._prev).__name__ == "Node": curr._prev._next = curr
        # for if if
        self.tail = curr
    # __init__
    
    def __call__(self, x):
        def propagate(curr, buffer_result):
            if curr._next is None: return curr(buffer_result)
            return propagate(curr._next, curr(buffer_result))
        # propagate
        return propagate(self.root, x)
    # forward
    
    def train(self, dataset, iters: int, lr=0.01, act_func=relu):
        def backpropagate(curr, x, y):
            if curr._prev is None: # root
                curr.train(x, curr.output)
                return
            if curr._next is None: # tail
                curr.train(curr._prev.output, y)
                return backpropagate(curr._prev, x, y)
            curr.train(curr._prev.output, curr.output)
            return backpropagate(curr._prev, x, y)
        # backpropagation
        for _ in range(iters):
            for x, y in dataset:
                self.__call__(x)
                backpropagate(self.tail, x, y)
        # for for
    # train
    
    def loss(self, dataset):
        x, y = dataset.x, dataset.y
        probability = self.__call__(x)
        loss = np.mean(np.exp(1 - probability) * y) + np.sum(np.exp(probability) * (1 - y))
        print(f"cost: {loss:.3f}")
    # loss
# ANN

In [7]:
class DNN(ANN):
    def __init__(self, n_inpt, n_output):
        self.l1 = Node(n_inpt, 2)
        self.l2 = Node(2, 2, relu)
        self.l3 = Node(2, n_output, sigmoid)
        super().__init__()
    # __init__
# DNN

In [8]:
model = DNN(len(trainset.x[0]), len(trainset.y[0]))
model.loss(trainset)
model.train(trainset, iters=1000, lr=0.01)

AttributeError: 'Node' object has no attribute 'ouput'

In [ ]:
# init a testset
index = random.randrange(0, 150)
test_x, test_y = testset[index]

discrimination = lambda arr, value=1: np.where(arr == value)[0]

labels = ["setosa","versicolour","virginica"]
setosa = np.array([list(trainset.x[_]) for _ in range(trainset.__len__()) if discrimination(trainset.y[_], 1) == 0])
versicolour = np.array([list(trainset.x[_]) for _ in range(trainset.__len__()) if discrimination(trainset.y[_], 1) == 1])
virginica = np.array([list(trainset.x[_]) for _ in range(trainset.__len__()) if discrimination(trainset.y[_], 1) == 2])
pred = model(test_x)

fig, ax1 = plt.subplots(1, 1, figsize=(12, 6))
ax1.set_xlabel('sepal length')
ax1.set_ylabel('petal length')
ax1.grid(True, alpha=0.5)

ax1.scatter(x=setosa[:, 2], y=setosa[:, 1], s=50, alpha=0.8, c="violet", label="setosa")
ax1.scatter(x=versicolour[:, 2], y=versicolour[:, 1], s=50, alpha=0.8, c="pink", label="versicolour")
ax1.scatter(x=virginica[:, 2], y=virginica[:, 1], s=50, alpha=0.8, c="skyblue", label="virginica")
ax1.scatter(x=test_x[0], y=test_x[1], s=100, alpha=1, c="r", marker="+", label=f"prediction, '{labels[np.argmax(pred)]}'")
ax1.legend(fontsize="10", loc="best", title="Iris Prediction")

In [ ]:
model.loss(trainset)